In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Food Order Analysis").getOrCreate()

In [0]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DateType
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
salesSchema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("customer_id", StringType(), True),
    StructField("order_date", DateType(), True),
    StructField("location", StringType(), True),
    StructField("vendor", StringType(), True)
])

In [0]:
sales_df = spark.read.schema(salesSchema).format('csv').option('inferschema', 'true').load("/FileStore/tables/foodOrder/sales.csv")

In [0]:
sales_df.show(20)

+--------+-----------+----------+--------+----------+
|order_id|customer_id|order_date|location|    vendor|
+--------+-----------+----------+--------+----------+
|       1|          A|2023-01-01|   India|    Swiggy|
|       2|          A|2022-01-01|   India|    Swiggy|
|       2|          A|2023-01-07|   India|    Swiggy|
|       3|          A|2023-01-10|   India|Restaurant|
|       3|          A|2022-01-11|   India|    Swiggy|
|       3|          A|2023-01-11|   India|Restaurant|
|       2|          B|2022-02-01|   India|    Swiggy|
|       2|          B|2023-01-02|   India|    Swiggy|
|       1|          B|2023-01-04|   India|Restaurant|
|       1|          B|2023-02-11|   India|    Swiggy|
|       3|          B|2023-01-16|   India|    zomato|
|       3|          B|2022-02-01|   India|    zomato|
|       3|          C|2023-01-01|   India|    zomato|
|       1|          C|2023-01-01|      UK|    Swiggy|
|       6|          C|2022-01-07|      UK|    zomato|
|       3|          D|2023-0

In [0]:
sales_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- location: string (nullable = true)
 |-- vendor: string (nullable = true)



In [0]:
menuSchema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("dish_name", StringType(), True),
    StructField("price", IntegerType(), True)
])

In [0]:
data = [
    (1, "PIZZA", 100),
    (2, "Chowmin", 150),
    (3, "Sandwich", 120),
    (4, "Dosa", 110),
    (5, "Biryani", 80),
    (6, "Pasta", 180)
]

menu2_df = spark.createDataFrame(data, menuSchema)

In [0]:
menu2_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- dish_name: string (nullable = true)
 |-- price: integer (nullable = true)



In [0]:
menu2_df.show(3)

+--------+---------+-----+
|order_id|dish_name|price|
+--------+---------+-----+
|       1|    PIZZA|  100|
|       2|  Chowmin|  150|
|       3| Sandwich|  120|
+--------+---------+-----+
only showing top 3 rows



In [0]:
#Adding month, year and quater into the sales_df dataframe
sales_df = sales_df.withColumn("order_year", year(sales_df.order_date))
sales_df = sales_df.withColumn("order_month", month(sales_df.order_date))
sales_df = sales_df.withColumn("order_quarter", quarter(sales_df.order_date))

In [0]:
sales_df.show(5)

+--------+-----------+----------+--------+----------+----------+-----------+-------------+
|order_id|customer_id|order_date|location|    vendor|order_year|order_month|order_quarter|
+--------+-----------+----------+--------+----------+----------+-----------+-------------+
|       1|          A|2023-01-01|   India|    Swiggy|      2023|          1|            1|
|       2|          A|2022-01-01|   India|    Swiggy|      2022|          1|            1|
|       2|          A|2023-01-07|   India|    Swiggy|      2023|          1|            1|
|       3|          A|2023-01-10|   India|Restaurant|      2023|          1|            1|
|       3|          A|2022-01-11|   India|    Swiggy|      2022|          1|            1|
+--------+-----------+----------+--------+----------+----------+-----------+-------------+
only showing top 5 rows



In [0]:
#Total amount spent by each customer
total_amount_spent = (sales_df.join(menu2_df, 'order_id').groupBy('customer_id').agg({'price':'sum'}).orderBy('customer_id'))

In [0]:
display(total_amount_spent)

customer_id,sum(price)
A,4260
B,4440
C,2400
D,1200
E,2040


Databricks visualization. Run in Databricks to view.

In [0]:
#Total amount spent on each food item
total_amount_spent_on_food_item = (sales_df.join(menu2_df,'order_id').groupBy('dish_name').agg({'price':'sum'}).orderBy('dish_name'))

In [0]:
display(total_amount_spent_on_food_item)

dish_name,sum(price)
Biryani,480
Chowmin,3600
Dosa,1320
PIZZA,2100
Pasta,1080
Sandwich,5760


Databricks visualization. Run in Databricks to view.

In [0]:
#Total amount of sales in each month
Total_sales_in_months = (sales_df.join(menu2_df, 'order_id').groupBy('order_month').agg({'price':'sum'}).orderBy('order_month'))

In [0]:
display(Total_sales_in_months)

order_month,sum(price)
1,2960
2,2730
3,910
5,2960
6,2960
7,910
11,910


Databricks visualization. Run in Databricks to view.

In [0]:
#How many times each product purchased
most_product_purchase = (sales_df.join(menu2_df, 'order_id').groupBy('dish_name').agg(count('order_id').alias('order_count')).orderBy('order_count', ascending=0))

In [0]:
display(most_product_purchase)

dish_name,order_count
Sandwich,48
Chowmin,24
PIZZA,21
Dosa,12
Biryani,6
Pasta,6


Databricks visualization. Run in Databricks to view.

In [0]:
#Frequency of customers visited to resturant
frequency = (sales_df.filter(sales_df.vendor == 'Restaurant').groupBy('customer_id').agg(countDistinct('order_date').alias('number_of_distinct_order_per_customer')))

In [0]:
display(frequency)

customer_id,number_of_distinct_order_per_customer
E,5
B,6
D,1
C,3
A,6
